In [103]:
import requests
import folium
import polyline
import json


In [104]:

def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out


In [105]:

    
def get_map(route, line=True):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    if line:
        folium.PolyLine(
            route['route'],
            weight=8,
            color='blue',
            opacity=0.6
        ).add_to(m)
    else:
        for idx, marker in enumerate(route['route']):
            folium.Marker(
            location=(marker[1], marker[0]),
            icon=folium.Icon(icon='play', color=route['route_colors'][idx])
        ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m


In [106]:

pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = 19.8512,45.246,19.841270,45.253672

with open('route_visited.json', 'r') as j:
     r = json.loads(j.read())['arr']
     rt = r[0]
     rc = r[1]
test_route = {
    'route': rt,
    'start_point': (rt[0][1], rt[0][0]),
    'end_point': ((rt[-1][1], rt[-1][0])),
    'route_colors': rc
}
m=get_map(test_route, line=False)

In [107]:
test_route

{'route': [[19.8512, 45.246],
  [19.8512, 45.246],
  [19.85126, 45.24592],
  [19.85126, 45.24592],
  [19.85103, 45.24585],
  [19.85103, 45.24585],
  [19.8514, 45.24527],
  [19.8514, 45.24527],
  [19.850905, 45.245115],
  [19.85041, 45.24496],
  [19.85041, 45.24496],
  [19.85026, 45.24501],
  [19.85026, 45.24501],
  [19.850036666666664, 45.245358333333336],
  [19.849813333333334, 45.24570666666667],
  [19.84959, 45.246055],
  [19.849366666666665, 45.24640333333334],
  [19.84914333333333, 45.24675166666667],
  [19.84892, 45.2471],
  [19.84892, 45.2471],
  [19.84904, 45.24733],
  [19.84904, 45.24733],
  [19.84956, 45.24768],
  [19.84956, 45.24768],
  [19.849315714285716, 45.248045714285716],
  [19.84907142857143, 45.24841142857144],
  [19.848827142857147, 45.24877714285715],
  [19.84858285714286, 45.249142857142864],
  [19.848338571428574, 45.24950857142858],
  [19.84809428571429, 45.249874285714284],
  [19.84785, 45.25024],
  [19.84785, 45.25024],
  [19.848316, 45.250392],
  [19.848782, 

In [108]:
m